# Plotting live data from Cyton Board using Python and pyqtgraph


## Objective

1. Use the pyOpenBCI library to aquire data from the OpenBCI board and plot it in real time.
2. Process the OpenBCI data to get the different EEG bands (Alpha, Beta, etc..)


### Why use pyqtgraph?

From the pyqtgraph website:

"PyQtGraph is a graphics and user interface library for Python that provides functionality commonly required in engineering and science applications. Its primary goals are 1) to provide fast, interactive graphics for displaying data (plots, video, etc.) and 2) to provide tools to aid in rapid application development (for example, property trees such as used in Qt Designer).

For plotting, pyqtgraph is not nearly as complete/mature as matplotlib, but runs much faster. Matplotlib is more aimed toward making publication-quality graphics, whereas pyqtgraph is intended for use in data acquisition and analysis applications."

Because the Cyton Board data comes at a rate of 250Hz, a fast way to display data is needed and pyqtgraph is the way to go.


### Notes:
1. This notebook is being run on a Windows 10 Laptop, using a Python3 environment.

## First let's import the necessary packages:


In [6]:
import sys
sys.path.append('C:/Python37/Lib/site-packages')

from IPython.display import clear_output
%matplotlib qt5
%matplotlib inline


from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import random
from pyOpenBCI import OpenBCICyton
import threading
import time
import numpy as np

#### Now let's try to run the [test code](https://github.com/OpenBCI/pyOpenBCI) from the pyOpenBCI repo

We are doing this to make sure all the dependencies are installed correctly. When working this code should print the raw data of the board. If everything is working correctly you can interrupt the kernel to exit. You should see the Cyton data printing.

In [7]:
from pyOpenBCI import OpenBCICyton

def print_raw(sample):
    print(sample.channels_data)

board = OpenBCICyton(port='COM5', daisy=False)

board.start_stream(print_raw)

Serial established
Skipped 100 bytes before start found
[-8388608, -8376720, -8388608, -8388608, -8388608, -8388608, -8388608, -5092048]
[-8388608, -8376654, -8388608, -8388608, -8388608, -8388608, -8388608, -5091987]
[-8388608, -8376574, -8388608, -8388608, -8388608, -8388608, -8388608, -5091915]
[-8388608, -8376396, -8388608, -8388608, -8388608, -8388608, -8388608, -5091748]
[-8388608, -8376376, -8388608, -8388608, -8388608, -8388608, -8388608, -5091735]
[-8388608, -8376367, -8388608, -8388608, -8388608, -8388608, -8388608, -5091735]
[-8388608, -8376322, -8388608, -8388608, -8388608, -8388608, -8388608, -5091700]
[-8388608, -8376237, -8388608, -8388608, -8388608, -8388608, -8388608, -5091621]
[-8388608, -8376179, -8388608, -8388608, -8388608, -8388608, -8388608, -5091570]
[-8388608, -8376119, -8388608, -8388608, -8388608, -8388608, -8388608, -5091517]
[-8388608, -8376091, -8388608, -8388608, -8388608, -8388608, -8388608, -5091499]
[-8388608, -8375952, -8388608, -8388608, -8388608, -8

[-8388608, -8367672, -8388608, -8388608, -8388608, -8388608, -8388608, -5084652]
[-8388608, -8367554, -8388608, -8388608, -8388608, -8388608, -8388608, -5084551]
[-8388608, -8367519, -8388608, -8388608, -8388608, -8388608, -8388608, -5084537]
[-8388608, -8367352, -8388608, -8388608, -8388608, -8388608, -8388608, -5084392]
[-8388608, -8367166, -8388608, -8388608, -8388608, -8388608, -8388608, -5084230]
[-8388608, -8367019, -8388608, -8388608, -8388608, -8388608, -8388608, -5084106]
[-8388608, -8367028, -8388608, -8388608, -8388608, -8388608, -8388608, -5084134]
[-8388608, -8366895, -8388608, -8388608, -8388608, -8388608, -8388608, -5084024]
[-8388608, -8366802, -8388608, -8388608, -8388608, -8388608, -8388608, -5083950]
[-8388608, -8366697, -8388608, -8388608, -8388608, -8388608, -8388608, -5083866]
[-8388608, -8366639, -8388608, -8388608, -8388608, -8388608, -8388608, -5083832]
[-8388608, -8366603, -8388608, -8388608, -8388608, -8388608, -8388608, -5083820]
[-8388608, -8366577, -838860

[-8388608, -8356761, -8388608, -8388608, -8388608, -8388608, -8388608, -5076792]
[-8388608, -8356608, -8388608, -8388608, -8388608, -8388608, -8388608, -5076669]
[-8388608, -8356424, -8388608, -8388608, -8388608, -8388608, -8388608, -5076513]
[-8388608, -8356369, -8388608, -8388608, -8388608, -8388608, -8388608, -5076494]
[-8388608, -8356324, -8388608, -8388608, -8388608, -8388608, -8388608, -5076479]
[-8388608, -8356195, -8388608, -8388608, -8388608, -8388608, -8388608, -5076380]
[-8388608, -8356097, -8388608, -8388608, -8388608, -8388608, -8388608, -5076314]
[-8388608, -8356051, -8388608, -8388608, -8388608, -8388608, -8388608, -5076296]
[-8388608, -8355845, -8388608, -8388608, -8388608, -8388608, -8388608, -5076119]
[-8388608, -8355718, -8388608, -8388608, -8388608, -8388608, -8388608, -5076024]
[-8388608, -8355729, -8388608, -8388608, -8388608, -8388608, -8388608, -5076069]
[-8388608, -8355486, -8388608, -8388608, -8388608, -8388608, -8388608, -5075856]
[-8388608, -8355178, -838860

[-8388608, -8344681, -8388608, -8388608, -8388608, -8388608, -8388608, -5068916]
[-8388608, -8344597, -8388608, -8388608, -8388608, -8388608, -8388608, -5068872]
[-8388608, -8344526, -8388608, -8388608, -8388608, -8388608, -8388608, -5068839]
[-8388608, -8344397, -8388608, -8388608, -8388608, -8388608, -8388608, -5068750]
[-8388608, -8344193, -8388608, -8388608, -8388608, -8388608, -8388608, -5068585]
[-8388608, -8344117, -8388608, -8388608, -8388608, -8388608, -8388608, -5068549]
[-8388608, -8344036, -8388608, -8388608, -8388608, -8388608, -8388608, -5068506]
[-8388608, -8343989, -8388608, -8388608, -8388608, -8388608, -8388608, -5068500]
[-8388608, -8343924, -8388608, -8388608, -8388608, -8388608, -8388608, -5068476]
[-8388608, -8343756, -8388608, -8388608, -8388608, -8388608, -8388608, -5068348]
[-8388608, -8343701, -8388608, -8388608, -8388608, -8388608, -8388608, -5068336]
[-8388608, -8343647, -8388608, -8388608, -8388608, -8388608, -8388608, -5068314]
[-8388608, -8343581, -838860

[-8388608, -8331669, -8388608, -8378970, -8388608, -8388608, -8388608, -5061104]
[-8388608, -8331461, -8388608, -8378765, -8388608, -8388608, -8388608, -5060941]
[-8388608, -8331344, -8388608, -8378652, -8388608, -8388608, -8388608, -5060869]
[-8388608, -8331316, -8388608, -8378632, -8388608, -8388608, -8388608, -5060895]
[-8388608, -8331251, -8388608, -8378567, -8388608, -8388608, -8388608, -5060875]
[-8388608, -8331116, -8388608, -8378436, -8388608, -8388608, -8388608, -5060785]
[-8388608, -8330922, -8388608, -8378241, -8388608, -8388608, -8388608, -5060637]
[-8388608, -8330842, -8388608, -8378163, -8388608, -8388608, -8388608, -5060609]
[-8388608, -8330646, -8388608, -8377972, -8388608, -8388608, -8388608, -5060461]
[-8388608, -8330384, -8388608, -8377712, -8388608, -8388608, -8388608, -5060245]
[-8388608, -8330214, -8388608, -8377542, -8388608, -8388608, -8388608, -5060117]
[-8388608, -8330132, -8388608, -8377463, -8388608, -8388608, -8388608, -5060085]
[-8388608, -8330106, -838860

[-8388608, -8317376, -8388608, -8365038, -8388608, -8388608, -8388608, -5052934]
[-8388608, -8317176, -8388608, -8364839, -8388608, -8388608, -8388608, -5052789]
[-8388608, -8317088, -8388608, -8364756, -8388608, -8388608, -8388608, -5052751]
[-8388608, -8316905, -8388608, -8364582, -8388608, -8388608, -8388608, -5052630]
[-8388608, -8316922, -8388608, -8364598, -8388608, -8388608, -8388608, -5052701]
[-8388608, -8316773, -8388608, -8364453, -8388608, -8388608, -8388608, -5052603]
[-8388608, -8316575, -8388608, -8364259, -8388608, -8388608, -8388608, -5052459]
[-8388608, -8316502, -8388608, -8364189, -8388608, -8388608, -8388608, -5052437]
[-8388608, -8316401, -8388608, -8364092, -8388608, -8388608, -8388608, -5052388]
[-8388608, -8316414, -8388608, -8364107, -8388608, -8388608, -8388608, -5052454]
[-8388608, -8316401, -8388608, -8364094, -8388608, -8388608, -8388608, -5052497]
[-8388608, -8316145, -8388608, -8363839, -8388608, -8388608, -8388608, -5052298]
[-8388608, -8316104, -838860

KeyboardInterrupt: 

### Let's now add filters to process the data

#### What do we need?

1. A notch filter at 60Hz to filter out main power interferance. 
2. A highpass filter at 1Hz to 50Hz in order to remove the DC offset.
3. Get an FFT of the processed data.
4. Take an average of the FFT data to get the EEG bands.
5. Plot everything live using pyqtgraph.


In [3]:
SCALE_FACTOR = (4500000)/24/(2**23-1) #From the pyOpenBCI repo
colors = 'rgbycmwr'

app = QtGui.QApplication([])
win = pg.GraphicsWindow(title='Python GUI')
title_graph = win.addPlot(row=0, col=0, colspan=4,title='Python GUI')
ts_plots = [win.addPlot(row=i, col=0, colspan=2, title='Channel %d' % i) for i in range(1,9)]
fft_plot = win.addPlot(row=1, col=2, rowspan=4, title='FFT Plot')
fft_plot.setLimits(xMin=1,xMax=125, yMin=0, yMax=1e7)
waves_plot = win.addPlot(row=5, col=2, rowspan=4, title='Waves')
waves_plot.setLimits(xMin=0.5, xMax=5.5)
data = [[0,0,0,0,0,0,0,0]]


def save_data(sample):
    global data
    data.append([i*SCALE_FACTOR for i in sample.channels_data])

def updater():
    global data, plots, colors
    t_data = np.array(data[-1250:]).T #transpose data
    fs = 250 #Hz

    #Notch Filter at 60 Hz
    def notch_filter(val, data, fs=250):
        notch_freq_Hz = np.array([float(val)])
        for freq_Hz in np.nditer(notch_freq_Hz):
            bp_stop_Hz = freq_Hz + 3.0 * np.array([-1, 1])
            b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
            fin = data = signal.lfilter(b, a, data)
        return fin


    def bandpass(start, stop, data, fs = 250):
        bp_Hz = np.array([start, stop])
        b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
        return signal.lfilter(b, a, data, axis=0)

    nf_data = [[],[],[],[],[],[],[],[]]
    bp_nf_data = [[],[],[],[],[],[],[],[]]

    for i in range(8):
        nf_data[i] = notch_filter(60, t_data[i])
        bp_nf_data[i] = bandpass(15, 80, nf_data[i])

    for j in range(8):
        ts_plots[j].clear()
        ts_plots[j].plot(pen=colors[j]).setData(t_data[j])

    #fft of data
    fft_plot.clear()
    sp = [[],[],[],[],[],[],[],[]]
    freq = [[],[],[],[],[],[],[],[]]
    # for k in range(8):
    #     sp[k] = np.absolute(np.fft.fft(t_data[k]))
    #     freq[k] = np.fft.fftfreq(t_data[k].shape[-1], 1.0/fs)
    #     fft_plot.plot(pen=colors[k]).setData(freq[k], sp[k])

    for k in range(8):
        sp[k] = np.absolute(np.fft.fft(bp_nf_data[k]))
        freq[k] = np.fft.fftfreq(bp_nf_data[k].shape[-1], 1.0/fs)
        fft_plot.plot(pen=colors[k]).setData(freq[k], sp[k])


    # Define EEG bands
    eeg_bands = {'Delta': (1, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band (Only consider first channel)
    eeg_band_fft = dict()
    sp_bands = np.absolute(np.fft.fft(t_data[1]))
    freq_bands = np.fft.fftfreq(t_data[1].shape[-1], 1.0/fs)

    for band in eeg_bands:
        freq_ix = np.where((freq_bands >= eeg_bands[band][0]) &
                           (freq_bands <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(sp_bands[freq_ix])

    # bar [alpha, beta, theta]
    bg1 = pg.BarGraphItem(x=[1,2,3,4,5], height=[eeg_band_fft[band] for band in eeg_bands], width=0.6, brush='r')
    waves_plot.clear()
    waves_plot.addItem(bg1)


def start_board():
    board = OpenBCICyton(port='COM5', daisy=False)
    board.start_stream(save_data)

Now let's run the code we just added.

In [4]:
if __name__ == '__main__':
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        x = threading.Thread(target=start_board)
        x.daemon = True
        x.start()

        timer = QtCore.QTimer()
        timer.timeout.connect(updater)
        timer.start(0)


        QtGui.QApplication.instance().exec_()

Serial established
Skipped 11 bytes before start found
ID:<123> <Unexpected END_BYTE found <141> instead of <192>
Skipped 25 bytes before start found
Device appears to be stalling. Quitting...


The graph is a bit slow, but this is a great way to start to understand how to use the OpenBCI data for your application.

#### Signal Processing:

The next steps on your OpenBCI Python journey would be to do some signal processing. [Here](https://github.com/J77M/openbciGui_filter_test/blob/master/gui_saved_data_filter.ipynb) is a good example of some of the filters you can use with your OpenBCI data.